In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import astropy.io.fits as fits
import esutil

import lsst.afw.math as afwMath
import lsst.daf.persistence as dafPersist

ModuleNotFoundError: No module named 'esutil'

In [ ]:
# CCD selection
ccd = 43

In [ ]:
# butler
butlerVisit = dafPersist.Butler(
    '/datasets/hsc/repo/rerun/private/erykoff/fgcm_pdr1_run1wd/wide_deep2'
)
obsTable = butlerVisit.get(
    'fgcmVisitCatalog'
)
butler = dafPersist.Butler(
    '/datasets/hsc/repo/rerun/private/erykoff/hscproc/runIsrPDR1'
)

In [ ]:
# define amps
camera = butler.get('camera')
det = camera[ccd]
amp1, amp2, amp3, amp4 = det.getAmplifiers()

In [ ]:
# raw data bounding boxes
abox1 = amp1.getRawDataBBox()
abox2 = amp2.getRawDataBBox()
abox3 = amp3.getRawDataBBox()
abox4 = amp4.getRawDataBBox()

In [ ]:
# overscan bounding boxes
obox1 = amp1.getRawHorizontalOverscanBBox()
obox2 = amp2.getRawHorizontalOverscanBBox()
obox3 = amp3.getRawHorizontalOverscanBBox()
obox4 = amp4.getRawHorizontalOverscanBBox()

In [ ]:
# bounding box plot
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = (11,5)
plt.rcParams.update({'font.size': 14})
plt.suptitle('Bounding Boxes')
plt.title('purple = horizontal overscan, orange = data', fontsize=10)
plt.xlabel('x pixel position', fontsize=14)
plt.ylabel('y pixel position', fontsize=14)

# corners
allocorners = np.array([obox1.getCorners(), obox2.getCorners(), obox3.getCorners(), obox4.getCorners()])
allacorners = np.array([abox1.getCorners(), abox2.getCorners(), abox3.getCorners(), abox4.getCorners()])

# raw data vertices
for acorners in allacorners:
    x = [i[0] for i in acorners]
    y = [i[1] for i in acorners]
    x.append(x[0])
    y.append(y[0])
    abbox = plt.plot(x, y, color='#f1a340', label='data')
# overscan vertices
for ocorners in allocorners:
    x = [i[0] for i in ocorners]
    y = [i[1] for i in ocorners]
    x.append(x[0])
    y.append(y[0])
    obbox = plt.plot(x, y, color='#998ec3', label='horiz. overscan')

# finish up
#plt.legend(loc='upper center', fontsize=10, ncol=2)
plt.show()

In [ ]:
# bounding box plot
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.figure(figsize=(11, 5))
#plt.axis('equal')
plt.rcParams.update({'font.size': 14})
plt.suptitle('Bounding Boxes')
plt.title('purple = horizontal overscan, orange = data', fontsize=10)
plt.xlabel('x pixel position', fontsize=14)
plt.ylabel('y pixel position', fontsize=14)

# corners
allocorners = np.array([obox1.getCorners(), obox2.getCorners(), obox3.getCorners(), obox4.getCorners()])
allacorners = np.array([abox1.getCorners(), abox2.getCorners(), abox3.getCorners(), abox4.getCorners()])

# raw data vertices
for acorners in allacorners:
    x = [i[0] for i in acorners]
    y = [i[1] for i in acorners]
    plt.fill(x, y, color='#f1a340', alpha=0.85)
# overscan vertices
for ocorners in allocorners:
    x = [i[0] for i in ocorners]
    y = [i[1] for i in ocorners]
    plt.fill(x, y, color='#998ec3', alpha=0.85)

# finish up
plt.show()

In [ ]:
# band selection [all|g|r|i|z|y]
band = 'all'

In [ ]:
# obsTable data
filtername = [d['filtername'] for d in np.asarray(obsTable)]
if band != 'all': 
    goodfilters = [i for i,s in enumerate(filtername) if band in s]
    filtername = np.asarray(filtername)[goodfilters]
else: 
    goodfilters = range(len(filtername))
visits = obsTable['visit'][goodfilters]
exptime = obsTable['exptime'][goodfilters]
skybackground = obsTable['skybackground'][goodfilters]
ufname,ufcount = np.unique(filtername, return_counts=True)
print('\033[1m')
print('Number of visits:', visits.size)
print('Number of visits by filter:', dict(zip(ufname,ufcount)))
print('Number of pixel rows:', abox1.getHeight())
print('\033[0m')

In [ ]:
# exposure time hisogram
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = (11,5)
plt.rcParams.update({'font.size': 14})
plt.title('Exposure Times')
plt.xlabel('time / minutes', fontsize=14)
plt.ylabel('frequency', fontsize=14)
plt.xticks(np.arange(0, max(exptime) + 60, 60), np.arange(0, (max(exptime) + 60)/60, 1, dtype='i'))
binwidth = 10
plt.hist(exptime, bins=np.arange(0, max(exptime) + binwidth, binwidth), color='#f1a340', edgecolor='white')
plt.annotate('filter: ' + band, xy=(0.15,0.85), xycoords='figure fraction', ha='center')
plt.show()

In [ ]:
# sky background hisogram
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = (11,5)
plt.rcParams.update({'font.size': 14})
plt.title('Sky Background')
plt.xlabel('background level', fontsize=14)
plt.ylabel('frequency', fontsize=14)
#plt.xticks(np.arange(0, max(skybackground) + 60, 60), np.arange(0, (max(skybackground) + 60)/60, 1, dtype='i'))
plt.yscale('log', nonposy='clip')
binwidth = 500
plt.hist(skybackground, bins=np.arange(0, max(skybackground) + binwidth, binwidth), color='#f1a340', edgecolor='white')
plt.annotate('filter: ' + band, xy=(0.9,0.85), xycoords='figure fraction', ha='center')
plt.show()

In [ ]:
# catalogue, set up a numpy array, one entry for each visit, containing 9 named fields
cat = np.zeros(visits.size, dtype=[('amp1b', 'i4', (abox1.getHeight(), 5)),
                                   ('amp2a', 'i4', (abox2.getHeight(), 5)),
                                   ('amp3b', 'i4', (abox3.getHeight(), 5)),
                                   ('amp4a', 'i4', (abox4.getHeight(), 5)),
                                   ('overscan1', 'i4', (obox1.getHeight(), 2)),
                                   ('overscan2', 'i4', (obox2.getHeight(), 2)),
                                   ('overscan3', 'i4', (obox3.getHeight(), 2)),
                                   ('overscan4', 'i4', (obox4.getHeight(), 2)),
                                   ('visit', 'i4'),
                                   ('filtername', 'U1'),
                                   ('exptime', 'i4'),
                                   ('skybackground', 'i4')
                                  ])
cat['visit'][:] = visits
cat['filtername'][:] = filtername
cat['exptime'][:] = exptime
cat['skybackground'][:] = skybackground

In [ ]:
# temp
#for i,v in enumerate(visits):
for i, v in enumerate(visits[:101]):
    
    # counter
    if (i % 50) == 0:
        print('On %d of %d' % (i, visits.size))
    
    # try to use butler to get raw data
    try:
        raw = butler.get('raw', visit=int(visits[i]), ccd=ccd)
    except:
        continue
    
    overscan1 = raw.maskedImage[obox1].getArrays()[0].astype(np.int64)
    overscan2 = raw.maskedImage[obox2].getArrays()[0].astype(np.int64)
    overscan3 = raw.maskedImage[obox3].getArrays()[0].astype(np.int64)
    overscan4 = raw.maskedImage[obox4].getArrays()[0].astype(np.int64)
    
    data1 = raw.maskedImage[abox1].getArrays()[0].astype(np.int64)
    data2 = raw.maskedImage[abox2].getArrays()[0].astype(np.int64)
    data3 = raw.maskedImage[abox3].getArrays()[0].astype(np.int64)
    data4 = raw.maskedImage[abox4].getArrays()[0].astype(np.int64)
    
    cat['amp1b'][i, :] = data1[:, -5: ]
    cat['amp2a'][i, :] = data2[:, 0: 5]
    cat['amp3b'][i, :] = data3[:, -5: ]
    cat['amp4a'][i, :] = data4[:, 0: 5]
    
    cat['overscan1'][i, :, 0] = overscan1[:, 0]
    cat['overscan1'][i, :, 1] = overscan1[:, -1]
    cat['overscan2'][i, :, 0] = overscan2[:, -1]
    cat['overscan2'][i, :, 1] = overscan2[:, 0]
    cat['overscan3'][i, :, 0] = overscan3[:, 0]
    cat['overscan3'][i, :, 1] = overscan3[:, -1]
    cat['overscan4'][i, :, 0] = overscan4[:, -1]
    cat['overscan4'][i, :, 1] = overscan4[:, 0]

fits.writeto('test_last5pix_ccd%03d.fits' % (ccd), cat, overwrite=True)